In [ ]:
import requests
import time
from tqdm import tqdm

### Exposing Local Servers with Ngrok
To test both the **serverless** and **server-based** deployments, you'll need to expose each of them to the public internet using Ngrok. Here's how to do it step by step:

1. **Ensure Both Deployments Are Running**  
   Make sure that both the **serverless** function and the **server-based** Flask API are running locally on their respective ports (e.g., `8080` for serverless, `5000` for the server).

2. **Expose Serverless with Ngrok**  
   Since the free plan of Ngrok only allows exposing one port at a time, we'll first expose the serverless function.
   
   Run the following command to expose the serverless function (running on port `8080`):
   ```bash
   ngrok http 8080
   ```
   Ngrok will provide a public URL (e.g., `http://<ngrok-subdomain>.ngrok.io`). Copy this URL.

3. **Update the Serverless URL**  
   In the notebook, replace the `SERVERLESS_URL` placeholder with the public URL provided by Ngrok.

4. **Run the Serverless Test**  
   Now you can run the performance test for the serverless function in the notebook.

5. **Stop Ngrok and Expose Server**  
   After completing the serverless test, stop the Ngrok process and expose the **server-based** API (running on port `5000`).

   Run the following command to expose the server:
   ```bash
   ngrok http 5000
   ```
   Ngrok will provide a new public URL for the server.

6. **Update the Server URL**  
   In the notebook, replace the `SERVER_URL` placeholder with the new URL provided by Ngrok.

7. **Run the Server Test**  
   Now you can run the performance test for the server-based deployment in the notebook.

In [ ]:
# Test configurations
SERVERLESS_URL = "https://<ngrok-forwarding-url>/function/image-classify" # Replace <ngrok-forwarding-url> with your own ngrok forwarding URL
SERVER_URL = "https://<ngrok-forwarding-url>/predict" # Replace <ngrok-forwarding-url> with your own ngrok forwarding URL
IMAGE_PATH = "./images/dog.jpg"
NUM_REQUESTS = 150
DURATION = 120  # seconds

In [ ]:
def test_latency(api_url):
    """Measure the latency of the API by sending multiple sequential requests and recording the response times."""
    latencies = []
    for _ in tqdm(range(NUM_REQUESTS), desc="Testing latency"):
        with open(IMAGE_PATH, 'rb') as img:
            start = time.time()
            response = requests.post(api_url, data=img)
            end = time.time()
            latencies.append(end - start)

    avg_latency = sum(latencies) / len(latencies)
    print(f"Average latency: {avg_latency:.4f} seconds")
    return latencies

In [ ]:
def test_throughput(api_url):
    """Test throughput by sending as many requests as possible within a given duration."""
    with open(IMAGE_PATH, 'rb') as img_file:
        img_data = img_file.read()

    request_count = 0
    start_time = time.time()
    end_time = start_time + DURATION

    with tqdm(total=DURATION, desc="Testing throughput", unit='s') as pbar:
        while time.time() < end_time:
            response = requests.post(api_url, data=img_data)
            if response.status_code == 200:
                request_count += 1
            elapsed_time = int(time.time() - start_time)
            pbar.n = elapsed_time
            pbar.refresh()
        pbar.refresh()

    throughput = request_count / DURATION
    print(f"Throughput: {throughput:.2f} requests/second")
    return throughput

In [ ]:
# Test serverless API
print("Testing serverless API...")
serverless_latencies = test_latency(SERVERLESS_URL)
serverless_throughput = test_throughput(SERVERLESS_URL)

In [ ]:
# Test server-based API
print("Testing server-based API...")
server_latencies = test_latency(SERVER_URL)
server_throughput = test_throughput(SERVER_URL)